In [1]:
import numpy as np
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from dotenv import load_dotenv
from langchain_chroma import Chroma
from uuid import uuid4

load_dotenv()

True

In [2]:
df = pd.read_csv("E:\JK Voice Assistant\dataset.csv")

In [3]:
df.head()

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights,image_links
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,NaN,Cricket Practice Net NYLON HDPE Material W x H...,https://rukminim1.flixcart.com/image/612/612/x...
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,₹152,₹600,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...,https://rukminim1.flixcart.com/image/612/612/x...
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,NaN,Cricket Rubber Ball Weight: 110 g,https://rukminim1.flixcart.com/image/612/612/x...
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,NaN,Cricket Synthetic Ball Weight: 110 g,https://rukminim1.flixcart.com/image/612/612/k...
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,₹329,"₹1,399",IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...,https://rukminim1.flixcart.com/image/612/612/j...


In [4]:
df.shape

(12041, 12)

In [5]:
df.drop(columns = ['image_links'], inplace=True)

In [6]:
df = df.dropna(subset = ['mrp'], how = 'all')

In [7]:
df['seller_rating'].fillna(value = 0, inplace=True)
df['product_rating'].fillna(value = 0, inplace=True)
df['seller_name'].fillna(value="", inplace=True)
df['description'].fillna(value="", inplace=True)
df['highlights'].fillna(value="", inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_32696\1380661829.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['seller_rating'].fillna(value = 0, inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_32696\1380661829.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [8]:
df.isna().sum()

category_1        0
category_2        0
category_3        0
title             0
product_rating    0
selling_price     0
mrp               0
seller_name       0
seller_rating     0
description       0
highlights        0
dtype: int64

In [9]:
df['id'] = range(1, len(df) + 1)


In [10]:
df

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights,id
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,,Cricket Practice Net NYLON HDPE Material W x H...,1
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,₹152,₹600,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...,2
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,,Cricket Rubber Ball Weight: 110 g,3
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,,Cricket Synthetic Ball Weight: 110 g,4
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,₹329,"₹1,399",IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...,5
...,...,...,...,...,...,...,...,...,...,...,...,...
12036,Home and Furniture,Pet Supplies,Fish and Aquatics,TWINS TWINS Wall Hanging Fish Bowl II Wall Mou...,3.9,₹380,₹999,Twins enterprises,3.9,Wall Hanging Fish Bowl with Free Stones\nHigh ...,,11662
12037,Home and Furniture,Pet Supplies,Fish and Aquatics,Nitishree Sugar Black Sand Gravel for Aquarium...,4.2,₹179,₹299,Nitishree pets,4.4,aquarium substrate can add a great contrast in...,Type: Sand Used For Salt Water and Fresh Water...,11663
12038,Home and Furniture,Pet Supplies,Fish and Aquatics,TAIYO Gold Toyko Turtle Food 1 Kg Chicken 1 kg...,4.4,₹354,₹600,AquaxPetProducts,4.9,Taiyo Turtle Food 1Kg Suitable for your Turtle...,For Turtle Flavor: Chicken Food Type: Dry Suit...,11664
12039,Home and Furniture,Pet Supplies,Fish and Aquatics,pcg Eastern Flavors Optimum Fish Food (importe...,4.4,₹498,₹599,Foodie Puppies Group,4.5,,For Fish Flavor: NA Food Type: Dry Suitable Fo...,11665


In [11]:
pattern = '^[a-zA-Z0-9.]*$'

# Filter rows where the 'A' column matches the pattern
filtered_df = df[df['description'].str.contains(pattern, regex=True)]
filtered_df

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights,id
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,,Cricket Practice Net NYLON HDPE Material W x H...,1
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,,Cricket Rubber Ball Weight: 110 g,3
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,,Cricket Synthetic Ball Weight: 110 g,4
5,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,,Cricket Synthetic Ball Weight: 110 g,6
6,"Sports, Books and More",Sports,Cricket,Ceat six hitman Bat with COVER Poplar Willow C...,3.5,₹329,"₹2,295",IndiaFit,4.7,,Age Group 15+ Yrs Blade Made of Poplar Willow ...,7
...,...,...,...,...,...,...,...,...,...,...,...,...
12020,Home and Furniture,Pet Supplies,Fish and Aquatics,Twins 233 Cylinder Aquarium Tank (1 L),3.9,₹342,"₹1,000",Twins enterprises,3.9,,,11646
12023,Home and Furniture,Pet Supplies,Fish and Aquatics,Virya 8 L Fish Bowl,4.0,₹299,"₹1,299",H_HOMES,4.1,,,11649
12028,Home and Furniture,Pet Supplies,Fish and Aquatics,TWINS TWINS Wall Mounted Fish Bowl II Wall Han...,3.9,₹380,₹999,Twins enterprises,3.9,,,11654
12032,Home and Furniture,Pet Supplies,Fish and Aquatics,Power Stone Gravel Unplanted Substrate (Black...,4.3,₹199,₹399,StoneKaart,3.8,,Type: Gravel Used For Salt Water and Fresh Wat...,11658


In [12]:
documents = []

for index, row in df.iterrows():
    description = Document(
        page_content = f"The category of this product is {row['category_1']} and subcategories are {row['category_2']} and {row['category_3']}. The title of the product is '{row['title']}' with a product rating of {row['product_rating']} and a selling price of {row['selling_price']} and MRP {row['mrp']}). The product is sold by {row['seller_name']} with a seller rating of {row['seller_rating']}. Product description: {row['description']}. Highlights include: {row['highlights']}.",
        metadata={"id": f"{row['id']}"},
    )
    documents.append(description)

In [13]:
len(documents)

11666

In [14]:
documents[0]

Document(page_content="The category of this product is Sports, Books and More and subcategories are Sports and Cricket . The title of the product is 'ITWOSERVICES CRICKET NET 100X10 CRICKET NET NYLON HDPE Cricket Net\xa0\xa0(Green)' with a product rating of 4.4 and a selling price of ₹1,615 and MRP ₹4,000). The product is sold by I2SERVICES with a seller rating of 4.4. Product description: . Highlights include: Cricket Practice Net NYLON HDPE Material W x H x D: 10 x 10.", metadata={'id': '1'})

In [15]:
temp=documents[0:10]
temp_2 = documents[2000:2010]
temp_3 = documents[3000:3010]
temp_4 = documents[5000:5010]
temp_5= documents[7500:7510]
temp_6 =documents[10000:10010]


In [17]:
combined = temp+temp_2+temp_3+temp_4+temp_5+temp_6
len(combined)

60

In [18]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("inference_api_key"), model_name="sentence-transformers/all-MiniLM-l6-v2"
)

vector_store = Chroma(
    collection_name="product_collection",
    embedding_function=embeddings,
    persist_directory="E:\JK Voice Assistant\Temp11_chromadb_langchain",  # Where to save data locally, remove if not neccesary,
    collection_metadata={"hnsw:space": "cosine"}
)

# uuids = [str(uuid4()) for _ in range(len(temp_docs))]

vector_store.add_documents(documents=combined)

['782dc29c-f879-4759-9069-a60515318442',
 '736e0953-055f-4024-b870-18ae2a3c3a31',
 '22c96740-529c-418e-b7fe-8d19c417d27d',
 '07a3f924-27d9-4d4b-8d3d-205234c604c0',
 '234b0ce5-afea-49ac-a6fb-bb221bec5fd4',
 'a18e8522-4e73-4483-902d-cc1b84774b9a',
 '329adddd-dcef-43c5-b186-ae0f78a105a9',
 '097e5d93-d56b-40f8-8dfa-763854d6a02c',
 'f9a994ba-96ed-4716-963b-6cc20696ab6d',
 'be6d866d-4d67-430b-bf4a-6b66098f1ecd',
 '648bced9-b3e1-4582-abc1-dea1ff293d55',
 '8c2d5abf-2a79-425f-ad0d-b1fb26b6a28e',
 'eb44814d-1f12-434f-9edc-9a9312a38071',
 '1891395a-eebe-44ac-9485-5aae1d7f9efd',
 '1801fb76-6241-4e14-a81b-580fa5001000',
 '3c731387-1de9-4dce-8a47-fc00e2bf7739',
 'fa5bd56e-f480-4441-84ef-b401b170fe4e',
 'adad68c4-53a0-401a-93e2-b70bda36632a',
 '18ad3a04-cac7-4dd6-a951-bf09e8c8f2eb',
 '2159fc39-af03-47de-82b4-92d10b0841a8',
 '625c25a2-243d-433d-b2d0-96304ef2161b',
 '131568f4-30cf-4591-ab59-c94875b37e3e',
 '13bada01-aa36-4ad7-bba8-de9e282cf2ed',
 'c7822728-085e-44b4-9470-8b9f8573f23f',
 '9fc03461-b25a-

In [9]:
vector_store.get()

{'ids': ['042809b8-439f-4839-8f04-6ec698ae9c2c',
  '09eca564-22c3-4f49-912a-749ff027f720',
  '0c80a1b9-9e08-4c50-bbd7-16ef9ecd8bba',
  '0ee5f3cd-c7b2-44d6-91a4-3646320ec6ba',
  '150b6dcd-2c28-41d5-b60a-4fc965daf9f8',
  '177803c4-3a85-4d70-9240-053c9be5558f',
  '1a9184ed-e283-4439-b476-06a44afdc4ee',
  '1d92b1ff-2ede-4297-9434-595933503f31',
  '2229ccba-7555-444d-abc1-a66667370ad3',
  '2299b908-ef6d-49a0-bfe6-15c3ee546174',
  '2571e4b8-1625-44bb-849a-4b7812c6aca2',
  '2c87823e-5dcd-412f-b8af-d3229150e7e2',
  '3051ee60-2983-4e1d-83ca-d30e00a3e604',
  '31644a25-7a43-4784-9067-94afe68461a6',
  '321bafae-05d4-41b0-b248-8b1ca01c55bc',
  '33dc7bf8-45ca-4ac8-866e-f8744304623d',
  '35f6dc4c-53de-42b3-9469-10918f8ea7e7',
  '364d0f34-a556-4c7c-a9cf-88cc73080b07',
  '3984076d-7dbc-477b-8d19-de6091a4664d',
  '3a4dd923-53c5-4bc2-8e00-36700d3b451b',
  '3a5f3ca0-c580-4007-8435-c5d72e1b1590',
  '3e28ea09-f158-46bc-9cf8-64917d372340',
  '409033d9-c682-465b-845f-b77d56fd765e',
  '41b3ff14-d52b-4b75-8d30-

In [ ]:
results = vector_store.similarity_search_by_vector(
    embedding=embeddings.embed_query("I want cricket bat and net"), k=2
)

In [22]:
import os
from uuid import uuid4
import pandas as pd
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma

# Example documents
documents = [
    {"page_content": "This is the first document.", "metadata": {"id": 1}},
    {"page_content": "This is the second document.", "metadata": {"id": 2}},
    # Add more documents as needed
]

# Embeddings function using HuggingFaceInferenceAPIEmbeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("inference_api_key"), model_name="sentence-transformers/all-MiniLM-l6-v2"
)

# Initialize Chroma vector store
vector_store = Chroma(
    collection_name="product_collection",
    embedding_function=embeddings,
    persist_directory="E:\\JK Voice Assistant\\Temp1_chromadb_langchain",
    collection_metadata={"hnsw:space": "cosine"}
)

# Generate UUIDs for each document
uuids = [str(uuid4()) for _ in range(len(documents))]

# Print documents and UUIDs for debugging
print("Documents:", documents)
print("UUIDs:", uuids)

# Ensure the length of documents matches the length of UUIDs
assert len(documents) == len(uuids), "The number of documents and UUIDs must match."

# Extract texts and metadatas from documents
texts = [doc["page_content"] for doc in documents]
metadatas = [doc["metadata"] for doc in documents]

# Add texts to vector store with corresponding UUIDs
try:
    vector_store.add_texts(texts=texts, metadatas=metadatas, ids=uuids)
    print("Documents added successfully.")
except KeyError as e:
    print(f"KeyError encountered: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


Documents: [{'page_content': 'This is the first document.', 'metadata': {'id': 1}}, {'page_content': 'This is the second document.', 'metadata': {'id': 2}}]
UUIDs: ['19c1afb6-0d3f-4183-9fb6-04e310c7ae84', '8c59ef84-694b-47bd-8ea2-59f854ee60f2']
Documents added successfully.
